# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
#define cities csv
weather_data = "output_data/cities.csv"

# Read cities csv
weather_data = pd.read_csv(weather_data)

# create dataframe from cities csv
weatherdf = pd.DataFrame(weather_data)
weatherdf.head()

,City,Lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,okhotsk,59.3833,143.3000,18.61,94.0,100.0,20.71,RU,1.612922e+09
1,bac lieu,9.2850,105.7244,80.87,73.0,84.0,13.00,VN,1.612922e+09
2,bafra,41.5678,35.9069,41.00,87.0,20.0,3.44,TR,1.612922e+09
3,fortuna,40.5982,-124.1573,50.00,93.0,90.0,9.22,US,1.612922e+09
4,ponta do sol,32.6667,-17.1000,57.99,70.0,70.0,1.99,PT,1.612922e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
#lat and lng locations
location = weatherdf[["Lat", "lng"]].astype(float)

humidity = weatherdf["Humidity"].astype(float)

In [74]:
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=1)

heat_layer = gmaps.heatmap_layer(location, weights=humidity,
                                dissipating=False, max_intensity=100,
                                point_radius = 1.5, opacity=1)

fig.add_layer(heat_layer)

plt.savefig("vacationheatmap.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [31]:
#narrow dataframe down to cities with ideal weather
ideal_weather_df = weather_data.loc[(weather_data["Wind Speed"] <10) & 
(weather_data["Cloudiness"] == 0) & (weather_data["Max Temp"] >70) &
(weather_data["Max Temp"] <80)].dropna()

ideal_weather_df.count()
ideal_weather_df.head()

,City,Lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
29,saint-philippe,-21.3585,55.7679,78.80,57.0,0.0,6.91,RE,1.612922e+09
30,new norfolk,-42.7826,147.0587,71.01,42.0,0.0,3.00,AU,1.612922e+09
68,arraial do cabo,-22.9661,-42.0278,74.16,85.0,0.0,5.88,BR,1.612922e+09
87,grand-lahou,5.1367,-5.0242,78.76,90.0,0.0,6.26,CI,1.612922e+09
162,saint-joseph,-21.3667,55.6167,78.80,57.0,0.0,6.91,RE,1.612922e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [36]:
# Create new dataframe
hotel_df = ideal_weather_df.loc[:,["City","Country", "Lat", "lng"]]

# Add Hotel Name column
hotel_df["Hotel Name"] = ""

hotel_df.count()
hotel_df.head()

,City,Country,Lat,lng,Hotel Name
29,saint-philippe,RE,-21.3585,55.7679,
30,new norfolk,AU,-42.7826,147.0587,
68,arraial do cabo,BR,-22.9661,-42.0278,
87,grand-lahou,CI,5.1367,-5.0242,
162,saint-joseph,RE,-21.3667,55.6167,


In [73]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

# print(params)

In [39]:

for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

# end search
print("-------End of Search-------")

Retrieving Results for Index 29: saint-philippe.
Closest hotel in saint-philippe is Les Embruns Du Baril.
------------
Retrieving Results for Index 30: new norfolk.
Closest hotel in new norfolk is Tynwald Willow Bend Estate.
------------
Retrieving Results for Index 68: arraial do cabo.
Closest hotel in arraial do cabo is Orlanova Boutique Hotel.
------------
Retrieving Results for Index 87: grand-lahou.
Closest hotel in grand-lahou is Hotel Beatitude de Grand Lahou.
------------
Retrieving Results for Index 162: saint-joseph.
Closest hotel in saint-joseph is Village Sauvage.
------------
Retrieving Results for Index 218: warrnambool.
Closest hotel in warrnambool is Deep Blue Hotel & Hot Springs.
------------
Retrieving Results for Index 225: koutiala.
Closest hotel in koutiala is Hotel Flasso 1.
------------
Retrieving Results for Index 229: colac.
Closest hotel in colac is Otway Gate Motel.
------------
Retrieving Results for Index 237: saint-pierre.
Closest hotel in saint-pierre is 

In [43]:
hotel_df.head()

,City,Country,Lat,lng,Hotel Name
29,saint-philippe,RE,-21.3585,55.7679,Les Embruns Du Baril
30,new norfolk,AU,-42.7826,147.0587,Tynwald Willow Bend Estate
68,arraial do cabo,BR,-22.9661,-42.0278,Orlanova Boutique Hotel
87,grand-lahou,CI,5.1367,-5.0242,Hotel Beatitude de Grand Lahou
162,saint-joseph,RE,-21.3667,55.6167,Village Sauvage


In [45]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "lng"]]

In [72]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))